In [13]:
import boto3
import sagemaker
import pandas as pd
from pyathena import connect

In [2]:

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [3]:
s3_private_data_path = "s3://{}/w2-musicData/csv".format(bucket)
print(s3_private_data_path)

s3://sagemaker-us-east-1-106006112223/w2-musicData/csv


In [4]:
%store s3_private_data_path

Stored 's3_private_data_path' (str)


In [5]:
!aws s3 cp "dataset.csv" $s3_private_data_path/

upload: ./dataset.csv to s3://sagemaker-us-east-1-106006112223/w2-musicData/csv/dataset.csv


In [8]:
!aws s3 ls $s3_private_data_path/

2024-09-15 03:43:03   20118244 dataset.csv


In [15]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [20]:
database_name = "w2-music-db"

# SQL statement to execute
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         track_id string,
         artists string,
         album_name string,
         track_name string,
         popularity int,
         duration_ms int,
         explicit bool,
         danceability decimal,
         energy decimal,
         key int,
         loudness decimal,
         mode int,
         speechiness decimal,
         acousticness decimal,
         instrumentalness decimal,
         liveness decimal,
         valence decimal,
         tempo decimal,
         time_signature int,
         track_genre string
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\t' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, table_name_tsv, s3_private_data_path
)

print(statement)

CREATE DATABASE IF NOT EXISTS w2-music-db


In [21]:
# Related resourcesimport pandas as pd
pd.read_sql(statement, conn)

/tmp/ipykernel_1931/1221829551.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)
Failed to execute query.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/pyathena/common.py", line 301, in _execute
    query_id = retry_api_call(
  File "/opt/conda/lib/python3.10/site-packages/pyathena/util.py", line 84, in retry_api_call
    return retry(func, *args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/tenacity/__init__.py", line 379, in __call__
    do = self.iter(retry_state=retry_state)
  File "/opt/conda/lib/python3.10/site-packages/tenacity/__init__.py", line 314, in iter
    return fut.result()
  File "/opt/conda/lib/python3.10/concurrent/futures/_base.py", line 451, in result
    return self.__get_result()
  File "/opt/conda/lib/python3.10/concurrent/futur

DatabaseError: Execution failed on sql: CREATE DATABASE IF NOT EXISTS w2-music-db
An error occurred (InvalidRequestException) when calling the StartQueryExecution operation: line 1:8: mismatched input 'DATABASE'. Expecting: 'MATERIALIZED', 'MULTI', 'OR', 'PROTECTED', 'ROLE', 'SCHEMA', 'TABLE', 'VIEW'
unable to rollback

In [19]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_1931/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws


In [ ]:
%%html

<p><b>Shutting down your kernel for this notebook to release resources.</b></p>
<button class="sm-command-button" data-commandlinker-command="kernelmenu:shutdown" style="display:none;">Shutdown Kernel</button>
        
<script>
try {
    els = document.getElementsByClassName("sm-command-button");
    els[0].click();
}
catch(err) {
    // NoOp
}    
</script>

In [ ]:
%%javascript

try {
    Jupyter.notebook.save_checkpoint();
    Jupyter.notebook.session.delete();
}
catch(err) {
    // NoOp
}